# TASK 1

In [1]:
import cv2
from matplotlib import pyplot as plt
import sys
import os
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

def weight_m(u, M, inverse): # use inverse = True when doing inverse Fourier Transform
    if inverse == False:
        W_m = np.full((M, 1), np.exp((-2 * np.pi * (0 + 1.j) * 1 * u) / M))
        for i in range(2, M + 1):
            col = np.full((M, 1), np.exp((-2 * np.pi * (0 + 1.j) * i * u) / M), dtype=complex)
            W_m = np.concatenate([W_m, col], axis=1)
        return W_m
    else : # inverse == True
        W_m = np.full((M, 1), np.exp((2 * np.pi * (0 + 1.j) * 1 * u) / M))
        for i in range(2, M + 1):
            col = np.full((M, 1), np.exp((2 * np.pi * (0 + 1.j) * i * u) / M), dtype=complex)
            W_m = np.concatenate([W_m, col], axis=1)
        return W_m

def weight_n(v, N, inverse): # use inverse = True when doing inverse Fourier Transform
    if inverse == False:
        W_n = np.full((1, N), np.exp((-2 * np.pi * (0 + 1.j) * 1 * v) / N))
        for i in range(2, N + 1):
            row = np.full((1, N), np.exp((-2 * np.pi * (0 + 1.j) * i * v) / N), dtype=complex)
            W_n = np.concatenate([W_n, row], axis=0)
        return W_n
    else : # inverse == True
        W_n = np.full((1, N), np.exp((-2 * np.pi * (0 + 1.j) * 1 * v) / N))
        for i in range(2, N + 1):
            row = np.full((1, N), np.exp((2 * np.pi * (0 + 1.j) * i * v) / N), dtype=complex)
            W_n = np.concatenate([W_n, row], axis=0)
        return W_n

def image_loader(img_path): # Image loader by passing the image path's parent directory as argument
    tmp_img_path_list = os.listdir(img_path)
    tmp_img_path_list.sort()
    start_img = tmp_img_path_list[-1]
    del tmp_img_path_list[-1]
    tmp_img_path_list.insert(0, start_img)
    # print(tmp_img_path_list)

    img_array_list = [] # append the loaded numpy arrays (images) here
    for img in tmp_img_path_list:
        tmp_path = os.path.join(img_path, img)
        img_arr = cv2.imread(tmp_path, cv2.IMREAD_GRAYSCALE) # Load image by using OPEN CV -> as numpy
        img_array_list.append(img_arr)
    # print(img_array_list)
    return img_array_list
    

# image_loader(img_path)


        

In [3]:
def formula_ft(img, startRow, startCol, M, N):

    temp_rows = []
        # Crop the image for reference
    img = img[startRow: startRow + M, startCol : startCol + N]
    for u in range(M): # for each row
        for v in range(N): # for each column
            W_m = weight_m(u, M, inverse=False)
            W_n = weight_n(v, N, inverse=False)
            # print(W_m.shape, img.shape, W_n.shape)
            temp_res = (W_m @ img) @ W_n # returns m x n
            temp_res = np.sum(temp_res)
            temp_res = (1/(M * N)) * temp_res * 100000
            temp_rows.append(temp_res)
    # temp_rows[0] = 0 # WTH
    temp_arr = np.multiply(np.array(temp_rows), 1/100000)
    result_arr = np.reshape(temp_arr, (-1, N))
    ft_val = result_arr

    return ft_val

def fourier_transform(img, patch_size, refcheck): # get the fourier transform result for each image of the images as list
    M, N = patch_size
    if img is None:
            print('[LOAD FAILURE]')
            sys.exit()

    if refcheck == True:
        startRow = 140
        startCol = 276
       
        # print('M, N', M, N)
        ft_val = formula_ft(img, startRow, startCol, M, N)
        
    else : # if listcheck == false
        # img = img / np.max(img)
        startRow = 0
        startCol = 0
        ft_val = formula_ft(img, startRow, startCol, M, N)
    
    return ft_val


# TASK 2 

In [2]:
def phase_correlation(ft_val_origin, to_compare, patch_size):
    before_inverse = np.multiply(ft_val_origin, np.conj(to_compare)) / np.absolute(np.multiply(ft_val_origin, np.conj(to_compare)))
    temp_rows = []
    # after_inverse = np.max(np.angle(before_inverse))
    M, N = patch_size # given as tuple (auto unboxing)
    # Inverse Fourier Transform
    for u in tqdm(range(M)):
        for v in range(N):
            W_m = weight_m(u, M, inverse=True)
            W_n = weight_n(v, N, inverse=True)
            temp_res = (W_m @ before_inverse) @ W_n # returns m x n
            temp_res = np.sum(temp_res)
            temp_res = temp_res * 100000
            temp_rows.append(temp_res)
    temp_result = np.array(temp_rows)
    after_inverse = np.reshape(temp_result, (-1, N))
    after_inverse = np.max(np.angle(after_inverse)) # Get the phase correlation result
    return after_inverse


In [4]:
# For parallel processing
def getcorrel_from_all_patches(img, ft_res, r_start, c_start, patch_size):
        tmp_img = img[r_start:r_start+patch_size[0], c_start:c_start+patch_size[1]] # Get the kernel sliding across the image
        tmp_img = fourier_transform(tmp_img, patch_size, refcheck=False)
        new_correlation_val = phase_correlation(ft_res, tmp_img, patch_size)
        return (new_correlation_val, r_start, c_start)
                

In [5]:
def main():
    img_path = '/home/carpedkm/GitHub/Signals_and_Systems/dataset (1)/'
    patch_size = (24, 30)
    img_list = image_loader(img_path)
    # for cnt, img_ in enumerate(img_list):
    #     img_list[cnt] = cv2.equalizeHist(img_)
    print('[IMAGE LOAD SUCCESSFUL]')
    ft_res = fourier_transform(img_list[0], patch_size, refcheck=True) # this is the FT for the referecne patch
    print('[FOURIER TRANSFORM OF THE REFERENCE PATCH SUCCESSFUL]')
    # FT Inverse transform
    M, N = patch_size
    temp_rows = []
    ft_res_temp = (np.abs(ft_res) - np.min(np.abs(ft_res))) / (np.max(np.abs(ft_res)) - np.min(np.abs(ft_res)))
    ft_res_temp = ft_res_temp.astype(float)
    plt.imshow(ft_res_temp)
    plt.show()
    print('[FOURIER END]')
    # Get the inverse fourier transfrom result of the reference patch
    for u in range(M):
        for v in range(N):
            W_m = weight_m(u, M, inverse=True)
            W_n = weight_n(v, N, inverse=True)
            temp_res = W_m @ ft_res @ W_n
            temp_res = np.sum(temp_res)
            temp_res = temp_res * 100000
            temp_rows.append(temp_res)
    temp_result = np.array(temp_rows)
    after_inverse = np.reshape(temp_result, (-1, N))
    after_inverse = (np.abs(after_inverse) - np.min(np.abs(after_inverse))) / (np.max(np.abs(after_inverse)) - np.min(np.abs(after_inverse)))
    after_inverse = after_inverse.astype(float)
    plt.imshow(after_inverse)
    plt.show()
    print('[INVERSE END]')
    buffer = patch_size
    # for test
    max_correl_loc = []
    for cnt, img in tqdm(enumerate(img_list)):
        r_loc = 0
        c_loc = 0
        print('[IMAGE', cnt, ']', 'Load start')
        phase_correlation_val = 0
        plt.imshow(img)
        plt.show()
        args = ((elem1, elem2) for elem1 in range(0, img.shape[0] - buffer[0], 8) for elem2 in range(0, img.shape[1] - buffer[1], 10))
        phases = Parallel(n_jobs=-1, backend='threading')(delayed(getcorrel_from_all_patches)(img, ft_res, r_start, c_start, patch_size)
            for r_start, c_start in args)
        phases_dict = {}
        for p, r, c in phases:
            phases_dict[p] = (r, c)

        key_comp = 0
        keystore = 0
        for key_ in list(phases_dict.keys()):
            if key_comp < np.angle(key_):
                keystore = key_
                key_comp = np.angle(key_)
        r_loc, c_loc = phases_dict[keystore]
        # for r_start in tqdm(range(0, img.shape[0] - buffer[0], 8)):
        #     for c_start in range(0, img.shape[1] - buffer[1], 10):
                
        #         tmp_img = img[r_start:r_start+patch_size[0], c_start:c_start+patch_size[1]] # Get the kernel sliding across the image
        #         tmp_img = fourier_transform(tmp_img, patch_size, refcheck=False)
        #         new_correlation_val = phase_correlation(ft_res, tmp_img, patch_size)
        #         if phase_correlation_val < new_correlation_val:
        #             phase_correlation_val = new_correlation_val
        #             r_loc = r_start
        #             c_loc = c_start
                
                # print(phase_correlation_val) # Get the temporary phase_correlation_val
        max_correl_loc.append((r_loc, c_loc))
        print('LOC :', r_loc, c_loc)
        img = cv2.rectangle(img, (c_loc, r_loc), (c_loc + patch_size[1], r_loc + patch_size[0]), color=(0, 0, 255), thickness=2)
        plt.imshow(img)
        # plt.imshow(img[r_loc:r_loc+patch_size[0], c_loc:c_loc+patch_size[1]])
        plt.show()


In [ ]:
main()